In [1]:
from tqdm.auto import tqdm
import boto3
import os
import time
from boto.mturk.connection import MTurkConnection
from boto.mturk.question import HTMLQuestion
import pandas as pd 
from boto.mturk.question import ExternalQuestion
from IPython.display import display, HTML
import matplotlib.pyplot as plt
plt.rc('font', size=14)
from importlib import reload
from boto.mturk.qualification import (
    Qualifications,
    PercentAssignmentsApprovedRequirement, 
    NumberHitsApprovedRequirement
)

import sys
sys.path.insert(0, '../scripts/')

import utils_mturk as um

from importlib import reload
reload(um)

/Users/alex/opt/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


<module 'utils_mturk' from '../scripts/utils_mturk.py'>

# Set up 

In [2]:
print(os.environ['AWS_ACCESS_KEY_ID'])
print(os.environ['AWS_SECRET_ACCESS_KEY'])

VN2M29BH4PCAJT9ABZKN
O9bcBpaprrXr6Q3dorn0XYI4Kp8go6oBDBYFYqeD


In [3]:
env = 'sandbox'
# env = 'production'
mturk = um.MTurkHandler(environment=env) #=production/sandbox

In [5]:
CUSTOM_QUALIFICATION = False # True
# sandbox editor qualification: 3H3KEN1OLUQQR02IYZSVMYM7ESCBIO
if False:
    sandbox_journalist_qual = mturk.create_qualification('journalist')
    production_journalist_qual = mturk.create_qualification('journalist')

`sandbox_journalist_qual`:


```{'QualificationType': {'QualificationTypeId': '3H3KEN1OLUQQR02IYZSVMYM7ESCBIO',
  'CreationTime': datetime.datetime(2021, 11, 4, 23, 0, 7, tzinfo=tzlocal()),
  'Name': 'journalist',
  'Description': 'A custom qualification group given to workers we deem good.',
  'Keywords': 'custom-group filtering',
  'QualificationTypeStatus': 'Active',
  'IsRequestable': True,
  'AutoGranted': False},
 'ResponseMetadata': {'RequestId': '28302a92-3fef-47d4-8f02-1b93f1e08258',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '28302a92-3fef-47d4-8f02-1b93f1e08258',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '315',
   'date': 'Fri, 05 Nov 2021 06:00:07 GMT'},
  'RetryAttempts': 0}}```
  
`production_journalist_qual`

```{'QualificationType': {'QualificationTypeId': '3YJP8DI8F7IJNZ5SWSN2GXBAZJF4Q2',
  'CreationTime': datetime.datetime(2021, 11, 4, 23, 3, 23, tzinfo=tzlocal()),
  'Name': 'journalist',
  'Description': 'A custom qualification group given to workers we deem good.',
  'Keywords': 'custom-group filtering',
  'QualificationTypeStatus': 'Active',
  'IsRequestable': True,
  'AutoGranted': False},
 'ResponseMetadata': {'RequestId': '0edf8c83-69c6-446a-8d20-930712b2efa7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0edf8c83-69c6-446a-8d20-930712b2efa7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '315',
   'date': 'Fri, 05 Nov 2021 06:03:22 GMT'},
  'RetryAttempts': 0}}```

In [6]:
worker_requirements = [
    ### number of hits approved
    {
        'QualificationTypeId': '000000000000000000L0',
        'Comparator': 'GreaterThanOrEqualTo',
        'IntegerValues': [80],
    },
    ## worker local
    {
        'QualificationTypeId': '00000000000000000071',
        'Comparator': 'EqualTo',
        'LocaleValues': [{
            "Country":"US",
        }],
        'RequiredToPreview': True,
    },
    ## percent assignments approved
    {
        'QualificationTypeId': '000000000000000000L0',
        'Comparator': 'GreaterThanOrEqualTo',
        'IntegerValues': [90],
    },
]

## custom qualification
if CUSTOM_QUALIFICATION:
    worker_requirements = []
    if env == 'production':
        worker_requirements.append({
#             'QualificationTypeId': '3WZ6PU0JYXSTA4EIPF2M6S1CMZ7KL8',
            'QualificationTypeId': '3YJP8DI8F7IJNZ5SWSN2GXBAZJF4Q2', # journalists
            'Comparator': 'GreaterThanOrEqualTo',
            'IntegerValues': [90],      
        })
    else:
        worker_requirements.append({
#             'QualificationTypeId': '381R35RGJFFV6VLBBDX2MLZFNSH414',  ## UCLA students
            'QualificationTypeId': '3H3KEN1OLUQQR02IYZSVMYM7ESCBIO',  ## journalists
#             '3FQWXCP5BDC6A66PD20NE8FM4G3H44' other workers
            'Comparator': 'GreaterThanOrEqualTo',
            'IntegerValues': [90],      
        })

# Launch Task

In [7]:
from jinja2 import Template, Environment, FileSystemLoader
import datetime
from IPython.display import display, HTML
jinja_env = Environment(loader=FileSystemLoader('../app/templates'))

In [8]:
ls ../app/templates/

table-annotation-affil-role.html       table-annotation.html
table-annotation-diversity.html        task-annotation-slim.html
table-annotation-slim-affil-role.html  task-annotation.html
table-annotation-slim-diversity.html   task-validation.html
table-annotation-slim-full.html


In [9]:
template = jinja_env.get_template('table-annotation-slim-diversity.html')
created_hits = []

In [30]:
import glob

In [31]:
files = glob.glob('../app/data/input_data/*/*')

In [33]:
all_output = []
for fname in files:
    with open(fname) as f:
        input_data = json.load(f)
        if 'html_data' in input_data:
            t = pd.DataFrame(input_data['html_data'])
            t['file'] = fname
            all_output.append(t)

In [36]:
all_data_df = pd.concat(all_output)

In [52]:
sources_to_annotate = (all_data_df
 .drop_duplicates(['head', 'file'])
 .dropna()['head']
 .value_counts()
 .loc[lambda s: s<=2]
 .pipe(lambda s: set(s.index))
)

In [60]:
import json

for fname in files:
    with open(fname) as f:
        output = json.load(f)
        
    if 'html_data' in output:
        html_data = output['html_data']
        source_heads = set(map(lambda x: x.get('head', ''), html_data))
        source_heads = list(filter(lambda x: x != '', source_heads))
        source_heads = list(filter(lambda x: x in sources_to_annotate, source_heads))
        if len(source_heads)> 0 :
            break

In [63]:
import copy

In [67]:
output_copy = copy.deepcopy(output)

In [68]:
output_copy['html_data'] = list(filter(lambda x: x.get('head', '') == source_heads[0], html_data))

In [76]:
source_idx = output_copy['html_data'][0]['source_idx']

In [80]:
output_fn = (fname
 .replace('input_data', 'output_data_diversity')
 .replace('to-annotate', 'annotated')
 .replace('.json', '__source-idx-%s.json' % source_idx)
)

In [83]:
i = 0
source = 't'
## make the HTML 
data = output_copy['html_data']
html = template.render(
    data=output_copy['html_data'],
    entry_id=output_copy['entry_id'],
    version=output_copy['version'],
    label=output_copy['label'],
    url=output_copy['url'],
    headline=output_copy['headline'],
    published_date=output_copy['published_date'],
    do_mturk=True,
    start_time=time.time(),
    output_fname=output_fn
)

## dump to disk for inspection
with open('../app/mturk/templated-question-example-%s.html' % i, 'w') as f:
    f.write(html)

if True:
    ## HTMLQuestion HIT
    description = '''
        We\'d like to match sentences from two edited versions of the same article. 
        Help us by drawing lines to connect blocks of text.
    '''
    title = 'Edit some news articles v2'
    new_hit = mturk.client.create_hit(
        Title = title,
        Description = description,
        Keywords = 'text, highlighting',
        Reward = '0.4',
        MaxAssignments = 1,
        LifetimeInSeconds = 17280000,
        AssignmentDurationInSeconds = 600000,
        AutoApprovalDelayInSeconds = 28800,
        Question = html,
        QualificationRequirements=worker_requirements #if env == 'production' else []
    )
    created_hits.append(new_hit)

ClientError: An error occurred (ParameterValidationError) when calling the CreateHIT operation: There was an error parsing the XML question or answer data in your request.  Please make sure the data is well-formed and validates against the appropriate schema. Details: The markup in the document following the root element must be well-formed. (1651022130297 s)